In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv('terrorismData.csv')
dataset.drop(['Summary','Target','Longitude','Latitude' ] ,axis=1,inplace=True)
dataset.rename({'Target_type':'Targettype' , 'Weapon_type':'Weapontype' }, axis=1 , inplace=True)

In [3]:
dataset['Targettype'] = dataset['Targettype'].replace(to_replace="Unknown" , value=np.nan)
dataset['Targettype'].fillna(dataset['Targettype'].mode()[0], inplace=True)

In [4]:
dataset['Weapontype'] = dataset['Weapontype'].replace(to_replace=["Unknown","Sabotage Equipment" , "Vehicle (not to include vehicle-borne explosives, i.e., car or truck bombs)" , "Fake Weapons" , "Radiological" , "Biological" , "Other"] , value=np.nan)
dataset['AttackType'] = dataset['AttackType'].replace(to_replace="Unknown" , value=np.nan)

In [5]:
mode_attacktype = dataset.pivot_table(values='AttackType' , columns='Targettype' , aggfunc=(lambda x : x.mode()[0]))
mode_weapontype = dataset.pivot_table(values='Weapontype' , columns='Targettype' , aggfunc=(lambda x : x.mode()[0]))
missing_Weapontype=dataset['Weapontype'].isnull()
missing_Attacktype=dataset['AttackType'].isnull()

In [6]:
dataset.loc[missing_Weapontype , 'Weapontype'] = dataset.loc[missing_Weapontype , 'Targettype'].apply(lambda x : mode_weapontype[x])
dataset.loc[missing_Attacktype , 'AttackType'] = dataset.loc[missing_Attacktype , 'Targettype'].apply(lambda x : mode_attacktype[x])

In [7]:
mean_killed = dataset.pivot_table(values='Killed' , columns='AttackType' , aggfunc=(lambda x : x.mean()[0]))
mean_killed = mean_killed.round(decimals=0)
mean_wounded = dataset.pivot_table(values='Wounded' , columns='AttackType' , aggfunc=(lambda x : x.mean()[0]))
mean_wounded = mean_wounded.round(decimals=0)

C:\Users\aarya\AppData\Local\Temp\ipykernel_15776\3461726633.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_killed = dataset.pivot_table(values='Killed' , columns='AttackType' , aggfunc=(lambda x : x.mean()[0]))
C:\Users\aarya\AppData\Local\Temp\ipykernel_15776\3461726633.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_wounded = dataset.pivot_table(values='Wounded' , columns='AttackType' , aggfunc=(lambda x : x.mean()[0]))


In [8]:
missing_attack = dataset['Killed'].isnull()
missing_wounded = dataset['Wounded'].isnull()

In [9]:
dataset.loc[missing_attack , 'Killed'] = dataset.loc[missing_attack , 'AttackType'].apply(lambda x : mean_killed[x])
dataset.loc[missing_wounded , 'Wounded'] = dataset.loc[missing_wounded , 'AttackType'].apply(lambda x : mean_wounded[x])

In [10]:
mode_state = dataset.pivot_table(values='State' , columns='Country' , aggfunc=(lambda x: x.mode()[0]))
missing_state = dataset['State'].isnull()
dataset.loc[missing_state , 'State'] = dataset.loc[missing_state , 'Country'].apply(lambda x : mode_state[x])

In [11]:
dataset['City'] = dataset['City'].replace(to_replace=["Unknown","unknown"] , value=np.nan)
missing_city = dataset['City'].isnull()
mode_city = dataset.pivot_table(values='City' , columns='State' , aggfunc=(lambda x:x.mode()))
dataset.loc[missing_city , 'City'] = dataset.loc[missing_city , 'State'].apply(lambda x: mode_city[x])

In [12]:
indexyear = dataset[(dataset['Year']<=2009)].index
dataset.drop(indexyear , inplace=True)
indexcountry = dataset[(dataset['Country']!='India')].index
dataset.drop(indexcountry , inplace=True)

In [13]:
unknowngroup = dataset[dataset['Group'] == 'Unknown'].index
knowngroup = dataset[dataset['Group'] != 'Unknown'].index
prediction = dataset.drop(knowngroup)
dataset = dataset.drop(unknowngroup)

In [14]:
import plotly.express as px
year1 = dataset.Year.value_counts()
years = pd.DataFrame({'year':year1.index , 'Number of attacks':year1.values})
years.sort_values(by = 'year' , inplace=True)
fg1 = px.line(years , x= 'year' , y = 'Number of attacks')
fg1.show()

In [15]:
dataset['Group'] = dataset['Group'].replace('Communist Party of India - Maoist (CPI-Maoist)','Maoists')
df1 = dataset.Group.value_counts()
df = pd.DataFrame({'group':df1.index , 'count':df1.values})
df.drop(df.index[5:] , axis = 0 , inplace=True)
fg2 = px.histogram(df , x = 'group' , y = 'count' , color='group' , title='Top 5 Terrorist Group')
fg2.show()

In [16]:
list = ["United Liberation Front of Assam (ULFA)" , "Maoists" , "Garo National Liberation Army" , "Hizbul Mujahideen (HM)","National Democratic Front of Bodoland (NDFB)"]
dataset= dataset[dataset.Group.isin(list) == True]

In [17]:
dataset.shape

(3211, 13)

In [18]:
state1 = dataset.State.value_counts()
state = pd.DataFrame({'state':state1.index , 'count':state1.values})
state.drop(state.index[10:] , axis=0 , inplace= True)
fg2 = px.histogram(state , x='state' , y='count' ,color='state', title="Top 10 State attacked by Terrorist")
fg2.show()

In [19]:
city1 = dataset.City.value_counts()
city = pd.DataFrame({'city':city1.index , 'count':city1.values})
city.drop(city.index[10:] , axis=0 , inplace= True)
fg3 = px.histogram(city , x='city' , y='count' ,color='city', title="Top 10 City attacked by Terrorist")
fg3.show()

In [20]:
attacktype1 = dataset.AttackType.value_counts()
attacktp = pd.DataFrame({'Attacktype':attacktype1.index , 'count':attacktype1.values})
fg4 = px.pie(attacktp , values='count' , names='Attacktype',title='Terrorist Attack Types')
fg4.show()

In [21]:
targettype1 = dataset.Targettype.value_counts()
ttdf = pd.DataFrame({'target type':targettype1.index , 'count':targettype1.values})
fg5 = px.histogram(ttdf , x = 'target type' , y = 'count' , color='target type' , title = 'Terrorist Target')
fg5.show()

In [22]:
weapontype1 = dataset.Weapontype.value_counts()
weapontp = pd.DataFrame({'weapontype':weapontype1.index , 'count':weapontype1.values})
fg6 = px.pie(weapontp , values='count' , names='weapontype',title='Terrorist Weapon Types')
fg6.show()

In [23]:
print(dataset.shape)

(3211, 13)


In [24]:
dataset.head()

,Year,Month,Day,Country,State,Region,City,AttackType,Killed,Wounded,Group,Targettype,Weapontype
94904,2010,1,7,India,Jharkhand,South Asia,Salpatra,Hostage Taking (Kidnapping),1.0,0.0,Maoists,Violent Political Party,Firearms
94905,2010,1,7,India,West Bengal,South Asia,Murabani,Hostage Taking (Kidnapping),1.0,0.0,Maoists,Terrorists/Non-State Militia,Firearms
94951,2010,1,15,India,Jharkhand,South Asia,Gumla,Bombing/Explosion,7.0,0.0,Maoists,Police,Explosives
94954,2010,1,15,India,Jharkhand,South Asia,Gumla district,Armed Assault,7.0,2.0,Maoists,Police,Firearms
94966,2010,1,17,India,Bihar,South Asia,Gaya,Bombing/Explosion,0.0,0.0,Maoists,Telecommunication,Explosives


In [25]:
dataset = dataset.drop(['Year' , 'Month' ,'Day','State','Region','City','Country'],axis=1)

In [26]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
dataset['AttackType'] = encoder.fit_transform(dataset['AttackType'])
dataset['Targettype'] = encoder.fit_transform(dataset['Targettype'])
dataset['Weapontype'] = encoder.fit_transform(dataset['Weapontype'])

In [27]:
from sklearn.model_selection import train_test_split
x = dataset.drop(['Group'] , axis = 1)
y = dataset['Group']
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size=0.2 , random_state=1)

In [28]:
acc_list = []
model_list = ["RandomForestClassifier","KNeighborsClassifier","DecisionTreeClassifier" ]

In [29]:
from sklearn.ensemble import RandomForestClassifier
clf1 = RandomForestClassifier(max_depth=4 , random_state=0)
model1 = clf1.fit(x_train , y_train)
acc_list.append(model1.score(x_test , y_test))

In [30]:
from sklearn.neighbors import KNeighborsClassifier 
clf2 = KNeighborsClassifier(n_neighbors=5 , metric='minkowski' , p=2)
model2 = clf2.fit(x_train , y_train)
acc_list.append(model2.score(x_test , y_test))

In [31]:
from sklearn.tree import DecisionTreeClassifier
clf3 = DecisionTreeClassifier()
model3 = clf3.fit(x_train , y_train)
acc_list.append(model3.score(x_test , y_test))

In [32]:
acc_df = pd.DataFrame(np.c_[model_list , acc_list] , columns=["Model","Accuracy"])

In [33]:
acc_df.head()

,Model,Accuracy
0,RandomForestClassifier,0.8320373250388803
1,KNeighborsClassifier,0.8164852255054432
2,DecisionTreeClassifier,0.80248833592535


In [34]:
prediction_dataset = prediction.copy()

In [35]:
prediction = prediction.drop(['Year' , 'Month' ,'Day','State','Region','City','Country' , 'Group'],axis=1)

In [36]:
prediction['AttackType'] = encoder.fit_transform(prediction['AttackType'])
prediction['Targettype'] = encoder.fit_transform(prediction['Targettype'])
prediction['Weapontype'] = encoder.fit_transform(prediction['Weapontype'])

In [37]:
predgrp = model3.predict(prediction)

In [38]:
prediction['Group'] = predgrp

In [42]:
prediction_dataset = prediction_dataset.drop(['Group'] , axis=1)
prediction_dataset['Group'] = predgrp
prediction_dataset.head()

,Year,Month,Day,Country,State,Region,City,AttackType,Killed,Wounded,Targettype,Weapontype,Group
94975,2010,1,17,India,Andhra pradesh,South Asia,Khammam,Armed Assault,1.0,0.0,Terrorists/Non-State Militia,Melee,Maoists
95013,2010,1,19,India,Manipur,South Asia,Imphal,Bombing/Explosion,0.0,2.0,Police,Explosives,Maoists
95015,2010,1,20,India,Manipur,South Asia,Imphal,Armed Assault,0.0,0.0,Private Citizens & Property,Firearms,Hizbul Mujahideen (HM)
95024,2010,1,21,India,Manipur,South Asia,Imphal,Bombing/Explosion,0.0,0.0,Private Citizens & Property,Explosives,Maoists
95027,2010,1,21,India,Manipur,South Asia,Bishenpur,Bombing/Explosion,0.0,0.0,Business,Explosives,Maoists
95029,2010,1,21,India,Manipur,South Asia,Bishnupur,Bombing/Explosion,0.0,0.0,Government (General),Explosives,Maoists
95045,2010,1,23,India,Assam,South Asia,Chirang district,Bombing/Explosion,0.0,0.0,Transportation,Explosives,Maoists
95056,2010,1,24,India,Orissa,South Asia,Koraput,Bombing/Explosion,0.0,2.0,Police,Explosives,Maoists
95062,2010,1,24,India,Assam,South Asia,Dibrugarh district,Bombing/Explosion,0.0,0.0,Private Citizens & Property,Explosives,Maoists
95064,2010,1,25,India,Assam,South Asia,Dhubri,Bombing/Explosion,0.0,0.0,Transportation,Explosives,Maoists


In [40]:
df2 = prediction_dataset.Group.value_counts()
df = pd.DataFrame({'group':df2.index , 'count':df2.values})
df.drop(df.index[5:] , axis = 0 , inplace=True)
fg2 = px.histogram(df , x = 'group' , y = 'count' , color='group' , title='Predicted Terrorist Group')
fg2.show()